# Analytical hertz solutions
It is often useful to have analytical hertz solutions to hand when solving tribology problems. Slippy contains two hertz solvers for tackling different types of problems. These are:

- solve\_hertz\_...
- hertz\_full

solve\_hertz\_line or solve\_hertz\_point can be used to find the remaing parameter when others are given. For example if the load required to give a certain maximum shear stress for a particular geometry and material pair is needed. 

hertz_full can be used when the geometry, materials and load are known, it gives more detailed results including functions for the stress in each body, when these are known.

These three functions are found in the contact sub package:

In [17]:
import slippy.contact as C

## solve_hertz_point
solve_hertz_point is used to find the remaining parameter given a set of parameters. For example it can find the load needed for a particular maximum pressure etc.. It cannot be used when there is an angle between the principal radii of the contacting bodies.

In [25]:
c.solve_hertz_point()

In [26]:
s.shape

(13, 13)

In [27]:
s.resample(0.049473900056532176)

In [28]:
s.shape

(128, 128)

In [29]:
s.profile[-1,-1]

0.7824503786937322

In [30]:
op[-1,-1]

0.7824503786937322

In [31]:
np.allclose(op, s.profile)

True

In [33]:
import slippy.contact as C

In [35]:
C.__dict__

{'__name__': 'slippy.contact',
 '__doc__': '\n===========================================================\nContact mechanics models (:mod:`slippy.contact`)\n===========================================================\n\n.. currentmodule:: slippy.contact\n\nThis module contains functions and classes for contact mechanics.\n\nAnalytical solutions to common problems\n=======================================\n\n.. autosummary::\n   :toctree: generated\n\n   hertz_full\n   solve_hertz_line\n   solve_hertz_point\n\nFunctions and classes for generating multistep contact models\n=============================================================\n\nFunctions for elastic BEM soultions assuming the small angle assumption is\nvalid\n\n.. autosummary::\n   :toctree: generated\n\n   ContactModel\n   Step\n   Material\n\nXXXXXXXX put some examples here!\n================================\n\n   \n',
 '__package__': 'slippy.contact',
 '__loader__': <_frozen_importlib_external.SourceFileLoader at 0x16c0b66da20